# Introduction

In Q-leraning, we maintain a matrix to save all the action values. However, this does not work for continuous space, or a complex environment with huge action and state spaces. It is natural to consider replace the Q-table by a function. In other words, we use a function to approximate Q. This is the core idea of DQN (Deep-Q Network).

Notice that DQN may not be suitable for continuous action space, since we will do $\max Q$ for the value updates, which could be hard for an NN if the actions are not discrete. 

First we write out the update rulws for Q-learning: $$Q(s, a) \leftarrow Q(s, a) + \alpha (r + \gamma \max_{a'}Q(s', a')- Q(s, a))$$

# CartPole env
In the CartPole Env, there exists a cart, on top of which stands a pole. The agent is required to move the cart horizontally such to make the pole standing. If the pole tilts too much, or the cart moves too far from the initial place, or neither of the two happenes for 200 frames, the game ends. The state of the agent is a tuple of (cart_position, cart_velocity, pole_angle, pole_tip_velocity). We have 2 actions, 0 for left and 1 for right.

# DQN for CartPole env

Instead of learning a function $f: (s, a) \rightarrow Q$, we can simply set it to learn the action given the state, i.e. $f: s \rightarrow a$. 

Now an improtant question comes to us: how do we define the loss function?

It is simple to use an MSE loss between $Q_\omega(s, a)$ (the learnt action value) and the TD estimation $r + \gamma \max_{a'}Q(s', a')$, which theoretically should equal the actaul action values:

$$\omega^* = \arg \min_{\omega} \frac{1}{2N}\sum_{i=1}^N[Q_\omega(s_i, a_i) - (r + \gamma \max_{a'}Q_\omega(s'_i, a'))]^2$$

Now that we have the loss function, we have expand RL into its NN form. Since DQN use the same idea as of Q-learning, it is also off-policy. Therefore, we can banalnce exploration and exploitation by $\epsilon$-greedy, and collect the sampled data for later updates. 

Before we implement DQN, there are 2 modules we need to know that facilitates DQN training to be stable and eminent, namely experience replay and target network. 

## Experience replay

Consider a supervised learning task, where we sample 1 or a batch of data for 1 gradient update. With the training goes on, especially when we have multiple epochs, a certain data sample is used multiple times. The reason why we can do this is that in supervised learning, we have an important assumption: the i.i.d assumption that says, all the data in the training set are independent from an identical distrituion. This is also the reason why RL, involving temporal data, could be hard to train using supervised methods: because the temporal struction breaks the i.i.d assumption. 

Since we use an NN to estimate $Q$, we will need much more data to feed it so that the network is thoroughly trained. This is why we need experience replay. 

In experience replay, we maintain a replay-loading area, where you will find all the sampled 4-element tuples $(s, a, r, s')$. In the training of the Q-network, we randomly sample from the area. By doing so, the training data satisfies i.i.d assumption, and each data sample can be used multiple times to train the network. 

## Target Network
The target of DQN training is to let $Q_\omega(s, a)$ approach $(r + \gamma \max_{a'} Q_\omega(s', a'))$. Since the TD error contains the output of the network, and the output of the network is changing during updates, it is very likely to induce an unstable training. In order to solve this issue, we introduce the target network. 

The idea of the target network is to fix the Q network for loss calculation. Thus, we have 2 sets of Q-networks:
- the previous Q-network for update, responsible for $Q_\omega(s, a)$ in loss calculation
- the target Q-network for stable loss calculation, responsible for $\max_{a'}Q_{\omega^-} (s', a')$

Every $C$ step the target $Q_{\omega^-}$ will be synchronized to $Q_\omega$, while $Q_\omega$ is updated every step based on gradient updating. 

To sum up, DQN algorithm is:

- Init $Q_\omega(s, a)$
- Copy $Q_\omega(s, a)$ to $Q_{\omega^-}(s, a)$
- Init replay pool $R$
- for episode $e \leftarrow 1$ to $E$:
    - get init state $s_1$
    - for timestep $t \leftarrow 1$ to $T$:
        - use $\epsilon$-greedy to choose action $a_t$
        - take $a_t$ and get the response $r_t, s_{t+1}$
        - put $(s_t, a_t, r_t, s_{t+1})$ into $R$
        - if $R$ has enough data, sample $N$ data $\{(s_i, a_i, r_i, s_{i+1})\}_{i=1, ..., N}$ from $R$
        - for each sampled data from $R$, calculate target $y_i = r_i + \gamma \max_{a'} Q_{\omega^-}(s_{i+1}, a')$
        - minimize loss $L = \frac{1}{N} \sum_{i} (y_i - Q_\omega(s_t, a_t))^2$, then update $Q_\omega$
        - update $Q_{\omega^-}$
    - end for
- end for

In [66]:
!pip install torch
!pip install gym --upgrade

In [72]:
!pip list | grep numpy

numpy                   2.2.5


In [73]:
import random
import numpy as np
np.bool8 = np.bool_
import gym
import collections
from tqdm import tqdm
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import rl_utils
import copy

In [92]:
class ReplayBuffer:
    ''' 经验回放池 '''
    def __init__(self, capacity):
        self.buffer = collections.deque(maxlen=capacity)  # 队列,先进先出

    def add(self, state, action, reward, next_state, terminated, truncated):  # 将数据加入buffer
        self.buffer.append((state, action, reward, next_state, terminated, truncated))

    def sample(self, batch_size):  # 从buffer中采样数据,数量为batch_size
        transitions = random.sample(self.buffer, batch_size)
        state, action, reward, next_state, terminated, truncated = zip(*transitions)
        return np.array(state), action, reward, np.array(next_state), terminated, truncated

    def size(self):  # 目前buffer中数据的数量
        return len(self.buffer)

In [93]:
class Qnet(torch.nn.Module):
    ''' 只有一层隐藏层的Q网络 '''
    def __init__(self, state_dim, hidden_dim, action_dim):
        super(Qnet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, action_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))  # 隐藏层使用ReLU激活函数
        return self.fc2(x)

In [119]:
class DQN:
    ''' DQN算法 '''
    def __init__(self, state_dim, hidden_dim, action_dim, learning_rate, gamma,
                 epsilon, target_update, device):
        self.action_dim = action_dim
        self.q_net = Qnet(state_dim, hidden_dim,
                          self.action_dim).to(device)  # Q网络
        # 目标网络
        self.target_q_net = copy.deepcopy(self.q_net).to(device)
        # 使用Adam优化器
        self.optimizer = torch.optim.Adam(self.q_net.parameters(),
                                          lr=learning_rate)
        self.gamma = gamma  # 折扣因子
        self.epsilon = epsilon  # epsilon-贪婪策略
        self.target_update = target_update  # 目标网络更新频率
        self.count = 0  # 计数器,记录更新次数
        self.device = device

    def take_action(self, state):  # epsilon-贪婪策略采取动作
        print(state)
        if np.random.random() < self.epsilon:
            action = np.random.randint(self.action_dim)
        else:
            state = torch.tensor([state], dtype=torch.float).to(self.device)
            action = torch.argmax(self.q_net(state), dim=1).numpy()[0]
        return action

    def update(self, transition_dict):
        # implement DQN
        states = torch.tensor(transition_dict["states"], dtype=torch.float).to(self.device)
        actions = torch.tensor(transition_dict["actions"], dtype=torch.float).view(-1, 1).to(self.device)
        rewards = torch.tensor(transition_dict["rewards"], dtype=torch.float).view(-1, 1).to(self.device)
        next_states = torch.tensor(transition_dict["next_states"], dtype=torch.float).to(self.device)
        terminated = torch.tensor(transition_dict["terminated"], dtype=torch.float).to(self.device)
        truncated = torch.tensor(transition_dict["truncated"], dtype=torch.float).to(self.device)

        max_q, _ = torch.max(self.target_q_net(next_states), dim=1)
        td_est = rewards + self.gamma * max_q * (1 - terminated) * (1 - truncated)
        q_values = self.q_net(states).gather(1, actions.long())
        dqn_loss = torch.mean(F.mse_loss(q_values, td_est)) # mean squared loss
        
        self.optimizer.zero_grad()
        dqn_loss.backward()
        self.optimizer.step()

        # update the target network if C steps passed
        if self.count % self.target_update == 0:
            self.target_q_net = self.target_q_net.load_state_dict(self.q_net.state_dict()) 
        self.count += 1

In [120]:
lr = 2e-3
num_episodes = 500
hidden_dim = 128
gamma = 0.98
epsilon = 0.01
target_update = 10
buffer_size = 10000
minimal_size = 500
batch_size = 64
device = torch.device("cuda") if torch.cuda.is_available() else torch.device(
    "cpu")

env_name = 'CartPole-v0'
env = gym.make(env_name)
random.seed(0)
np.random.seed(0)
# env.seed(0)
torch.manual_seed(0)
replay_buffer = ReplayBuffer(buffer_size)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
agent = DQN(state_dim, hidden_dim, action_dim, lr, gamma, epsilon,
            target_update, device)

return_list = []
for i in range(10):
    with tqdm(total=int(num_episodes / 10), desc='Iteration %d' % i) as pbar:
        for i_episode in range(int(num_episodes / 10)):
            episode_return = 0
            state = env.reset()[0]
            done = False
            while not done:
                print(state)
                action = agent.take_action(state)
                next_state, reward, terminated, truncated, info = env.step(action)
                replay_buffer.add(state, action, reward, next_state, terminated, truncated)
                state = next_state
                episode_return += reward
                # 当buffer数据的数量超过一定值后,才进行Q网络训练
                if replay_buffer.size() > minimal_size:
                    b_s, b_a, b_r, b_ns, b_ter, b_tru = replay_buffer.sample(batch_size)
                    transition_dict = {
                        'states': b_s,
                        'actions': b_a,
                        'next_states': b_ns,
                        'rewards': b_r,
                        'terminated': b_ter,
                        'truncated': b_tru
                    }
                    agent.update(transition_dict)
            return_list.append(episode_return)
            if (i_episode + 1) % 10 == 0:
                pbar.set_postfix({
                    'episode':
                    '%d' % (num_episodes / 10 * i + i_episode + 1),
                    'return':
                    '%.3f' % np.mean(return_list[-10:])
                })
            pbar.update(1)

/Users/jessiewang/Desktop/experiments/rl_learning/.venv/lib/python3.12/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
Iteration 0:   0%|          | 0/50 [00:00<?, ?it/s]/Users/jessiewang/Desktop/experiments/rl_learning/.venv/lib/python3.12/site-packages/gym/envs/classic_control/cartpole.py:177: UserWarning: WARN: You are calling 'step()' even though this environment has already returned terminated = True. You should always call 'reset()' once you receive 'terminated = True' -- any further steps are undefined behavior.
  logger.warn(
/var/folders/pb/f5kv730d7_j57wxqp_92dwhh0000gp/T/ipykernel_14852/4213218242.py:40: UserWarning: Using a target size (torch.Size([64, 64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  dqn_loss = torch.mean(F.mse_loss(q_v

[-0.01575167  0.01896739 -0.03422007 -0.01388554]
[-0.01575167  0.01896739 -0.03422007 -0.01388554]
[-0.01537232 -0.17564751 -0.03449778  0.2678072 ]
[-0.01537232 -0.17564751 -0.03449778  0.2678072 ]
[-0.01888527 -0.3702606  -0.02914164  0.5494127 ]
[-0.01888527 -0.3702606  -0.02914164  0.5494127 ]
[-0.02629048 -0.5649613  -0.01815338  0.83277345]
[-0.02629048 -0.5649613  -0.01815338  0.83277345]
[-0.03758971 -0.7598306  -0.00149791  1.1196923 ]
[-0.03758971 -0.7598306  -0.00149791  1.1196923 ]
[-0.05278632 -0.9549328   0.02089593  1.411905  ]
[-0.05278632 -0.9549328   0.02089593  1.411905  ]
[-0.07188498 -1.1503075   0.04913403  1.711046  ]
[-0.07188498 -1.1503075   0.04913403  1.711046  ]
[-0.09489113 -1.3459581   0.08335496  2.0186074 ]
[-0.09489113 -1.3459581   0.08335496  2.0186074 ]
[-0.12181029 -1.5418397   0.12372711  2.3358874 ]
[-0.12181029 -1.5418397   0.12372711  2.3358874 ]
[-0.15264708 -1.7378426   0.17044485  2.6639276 ]
[-0.15264708 -1.7378426   0.17044485  2.6639276 ]


TypeError: '_IncompatibleKeys' object is not callable